In [ ]:
import fbprophet as pr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
def get_covid_data():
    
    #get the latest data from OxCGRT
    DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
    full_df = pd.read_csv(DATA_URL,
                    parse_dates=['Date'],
                    encoding="ISO-8859-1",
                    dtype={"RegionName": str},
                    error_bad_lines=False)

    #add new cases and new deaths columns

    full_df['NewCases'] = np.nan
    full_df['NewDeaths'] = np.nan
    
    for state in full_df[(full_df['Jurisdiction'] == 'STATE_TOTAL')]['RegionName'].unique():
        state_inds = (full_df['Jurisdiction'] == 'STATE_TOTAL') & (full_df['RegionName'] == state)
        full_df.loc[state_inds, 'NewCases'] = full_df.loc[state_inds, 'ConfirmedCases'].diff().fillna(0)
        full_df.loc[state_inds, 'NewDeaths'] = full_df.loc[state_inds, 'ConfirmedDeaths'].diff().fillna(0)

    for country in full_df[(full_df['Jurisdiction'] == 'NAT_TOTAL')]['CountryName'].unique():
        nat_inds = (full_df['Jurisdiction'] == 'NAT_TOTAL') & (full_df['CountryName'] == country)
        full_df.loc[nat_inds, 'NewCases'] = full_df.loc[nat_inds, 'ConfirmedCases'].diff().fillna(0)
        full_df.loc[nat_inds, 'NewDeaths'] = full_df.loc[nat_inds, 'ConfirmedDeaths'].diff().fillna(0)

    return full_df

def get_region(df, division, region):
    if division == 'state':
        df = full_df[(full_df['Jurisdiction'] == 'STATE_TOTAL') & (full_df['RegionName'] == region)][:-1]
    elif division == 'country':
        df = full_df[(full_df['Jurisdiction'] == 'NAT_TOTAL') & (full_df['CountryName'] == region)][:-1]
    else: 
        print('Please specify "state" or "country" division')
    nans = df.isna().sum()
    columns = nans[nans <= 21].index
    df = df[columns]
    df.loc[df.Date == '2020-01-01', ['ConfirmedCases','ConfirmedDeaths']] = 0
    df = df.fillna(method = 'ffill')
    return df

def mean_percent_error(y_test, y_hat):
    error = np.abs(y_test - y_hat)
    percent_error = error/y_test
    mean_percent_error = percent_error.sum() / len(y_test)
    return mean_percent_error

In [ ]:
full_df = get_covid_data()
division = 'country'
region = 'United States'
df = get_region(df, division = division, region = region)
df.rename(columns = {'Date':'ds','NewCases':'y'}, inplace = True)
train_df = df[df.ds < '2020-12-01']
test_df = df[df.ds >= '2020-12-01']

In [ ]:
regressors = df.columns[:4]
keepers = []
errors = pd.DataFrame(columns = 'MAPE')
for column in regressors:
    m = pr.Prophet(seasonality_mode = 'multiplicative')
    m.add_country_holidays(country_name='US')
    for keeper in keepers:
        m.add_regressor(keeper)
    m.add_regressor(column)
    m.fit(train_df)
    future = m.make_future_dataframe(periods=len(test_df))
    forecast = m.predict(future)
    regressors = keepers + []
    errors.loc[keepers.] = mean_percent_error(test_df['y'].values, forecast['yhat'][-len(test_df):].values)


In [ ]:
forecast = m.predict(future)
prophet_mpse = mean_percent_error(test_df['y'].values, forecast['yhat'][-len(test_df):].values)
print(f'Prophet MPSE = {prophet_mpse}')
sarmimax_mpse = mean_percent_error(test_df['y'].values, yhat)
print(f'SARIMAX MPSE = {sarmimax_mpse}')

In [ ]:
prophet_mpse = mean_percent_error(test_df['y'].values, forecast['yhat'][-len(test_df):].values)
print(f'Prophet MPSE = {prophet_mpse}')
sarmimax_mpse = mean_percent_error(test_df['y'].values, yhat)
print(f'SARIMAX MPSE = {sarmimax_mpse}')

In [ ]:
model2 = SARIMAX(train_df[train_df['ds'] > '2020-03-31'].set_index('ds')['y'], order =(1,1,4), seasonal_order = (4,0,0,7),
                                       freq = 'D')
fit_model2 = model2.fit(maxiter = 200, disp = False)
yhat = fit_model2.forecast(len(test_df))


In [ ]:
m

In [ ]:
from fbprophet.plot import plot_plotly, plot_components_plotly

fig = plot_plotly(m, forecast, changepoints = True)
fig.add_trace(go.Scatter(x=test_df['ds'], y=test_df['y'], mode='markers'))
fig.show()

In [ ]:
plot_components_plotly(m, forecast)